In [81]:
import pandas as pd
import math
import numpy as np

In [2]:
telemetry = pd.read_csv("./Motorq Data Science Assignment/telemetry_data.csv")
triggers = pd.read_csv("./Motorq Data Science Assignment/triggers_soc.csv")
pnid = pd.read_csv("./Motorq Data Science Assignment/vehicle_pnid_mapping.csv")
arti_ign = pd.read_json("./Motorq Data Science Assignment/artificial_ign_off_data.json")

In [87]:
telemetry["TIMESTAMP"] = pd.to_datetime(telemetry["TIMESTAMP"], format="mixed")

In [89]:
telemetry

,Unnamed: 0,VEHICLE_ID,TIMESTAMP,SPEED,IGNITION_STATUS,EV_BATTERY_LEVEL,ODOMETER
0,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:45:42.018,NaN,NaN,NaN,NaN
1,1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:58:08.017,NaN,NaN,NaN,NaN
2,2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:57:02.031,NaN,NaN,NaN,38685.9
3,3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:38:57.002,0.0,on,NaN,NaN
4,4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 02:00:05.004,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1823547,749582,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:11:40.400,NaN,NaN,NaN,NaN
1823548,749583,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:11:41.014,NaN,NaN,NaN,NaN
1823549,749584,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:11:42.500,NaN,NaN,NaN,32072.7
1823550,749585,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:12:00.021,NaN,NaN,57.6,NaN


In [92]:
triggers["CTS"] = pd.to_datetime(triggers["CTS"], format="mixed")

In [137]:
triggers["PNID"] = triggers["PNID"].apply(str)

In [138]:
triggers

,Unnamed: 0,CTS,PNID,NAME,VAL
0,0,2021-12-01 17:58:27-05:30,206958332,IGN_CYL,OFF
1,1,2021-12-01 17:58:27-05:30,206958332,IGN_CYL,OFF
2,2,2021-12-01 17:56:24-05:30,206958332,IGN_CYL,ON
3,3,2021-12-01 17:56:24-05:30,206958332,CHARGE_STATE,50.1
4,4,2021-12-01 17:56:24-05:30,206958332,IGN_CYL,ON
...,...,...,...,...,...
68665,68665,2022-01-28 20:45:07-05:30,193290358,IGN_CYL,OFF
68666,68666,2022-01-28 20:45:52-05:30,207925262,IGN_CYL,ON
68667,68667,2022-01-28 20:45:52-05:30,207925262,CHARGE_STATE,49.0
68668,68668,2022-01-28 20:45:52-05:30,207925262,IGN_CYL,ON


In [94]:
triggers.isna().sum()

Unnamed: 0    0
CTS           0
PNID          0
NAME          0
VAL           0
dtype: int64

In [10]:
pnid

,Unnamed: 0,ID,IDS
0,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,"[\n ""256932585"",\n ""256932577"",\n ""25693258..."
1,1,ae912623-e122-4bfb-b2a1-7d407ea42b1b,NaN
2,2,39424065-b3be-45e5-8f7d-b7f16a6c190a,NaN
3,3,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,"[\n ""270199389"",\n ""270199393"",\n ""27019938..."
4,0,9893c80d-f282-46cf-a794-280f0306c936,"[\n ""256933093"",\n ""256933081"",\n ""25693308..."
5,1,56d8ca94-9b18-41d1-831f-7afd905326d4,"[\n ""331750017"",\n ""331750025"",\n ""33175002..."
6,2,026af092-a01e-4b0a-af38-f459725dabc6,NaN
7,3,654f3d0f-f2f9-4fc6-856d-91337bef92f7,NaN
8,4,ae912623-e122-4bfb-b2a1-7d407ea42b1b,NaN
9,5,39424065-b3be-45e5-8f7d-b7f16a6c190a,NaN


In [58]:
pnid.columns

Index(['Unnamed: 0', 'ID', 'IDS'], dtype='object')

In [97]:
arti_ign["timestamp"] = pd.to_datetime(arti_ign["timestamp"], format="mixed")

In [98]:
arti_ign

,vehicleId,timestamp,type
0,3e937810-3a3b-48d6-80ad-56ae12c3bed8,2022-01-31 19:13:06.001000+00:00,artificial_event
1,3e937810-3a3b-48d6-80ad-56ae12c3bed8,2022-01-31 04:25:05.011000+00:00,artificial_event
2,460a67ab-ee58-4279-b96d-ac56ac029d3e,2022-01-30 01:58:57.004000+00:00,artificial_event
3,9893c80d-f282-46cf-a794-280f0306c936,2022-01-30 21:43:42.027000+00:00,artificial_event
4,fb20e847-d2a1-4583-b874-393c9d1f3db9,2022-01-31 04:25:05.038000+00:00,artificial_event
...,...,...,...
406,f0704100-56e6-4117-ae4f-cf97e5d2d132,2021-12-19 07:32:14.004000+00:00,artificial_event
407,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 05:54:05.037000+00:00,artificial_event
408,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 04:10:51.041000+00:00,artificial_event
409,460a67ab-ee58-4279-b96d-ac56ac029d3e,2021-12-18 22:45:17.030000+00:00,artificial_event


In [100]:
arti_ign.isna().sum()

vehicleId    0
timestamp    0
type         0
dtype: int64

In [105]:
pnid2 = pd.DataFrame({
    "sno":[],
    "ID":[],
    "PNID":[]
})
for i in range(pnid.shape[0]):
    ins = pnid.iloc[i]
    ids = ins["IDS"]
    if isinstance(ids, str):
        ids = list(ids.replace("\n","").replace("[","").replace("]","").replace("\"","").split(","))
        for id in ids:
            pnid2 = pd.concat([pnid2, pd.DataFrame({"sno":ins["Unnamed: 0"],"ID": ins['ID'],"PNID": id}, index=[i])])
    else:
        pnid2 = pd.concat([pnid2, pd.DataFrame({"sno":ins["Unnamed: 0"],"ID": ins['ID'],"PNID": pd.NA}, index=[i])])
pnid2

,sno,ID,PNID
0,0.0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932585
0,0.0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932577
0,0.0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932581
0,0.0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932589
0,0.0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256782425
...,...,...,...
18,14.0,04105a12-59b9-447b-865f-599f48eed1d7,364272761
18,14.0,04105a12-59b9-447b-865f-599f48eed1d7,278608654
18,14.0,04105a12-59b9-447b-865f-599f48eed1d7,364272749
18,14.0,04105a12-59b9-447b-865f-599f48eed1d7,364272753


In [133]:
pnid2.duplicated().value_counts()

False    165
Name: count, dtype: int64

In [126]:
pnid2["ID"].nunique()

16

In [68]:
pnid2.isna().sum()

sno     0
ID      0
PNID    6
dtype: int64

In [11]:
arti_ign

,vehicleId,timestamp,type
0,3e937810-3a3b-48d6-80ad-56ae12c3bed8,2022-01-31 19:13:06.001000+00:00,artificial_event
1,3e937810-3a3b-48d6-80ad-56ae12c3bed8,2022-01-31 04:25:05.011000+00:00,artificial_event
2,460a67ab-ee58-4279-b96d-ac56ac029d3e,2022-01-30 01:58:57.004000+00:00,artificial_event
3,9893c80d-f282-46cf-a794-280f0306c936,2022-01-30 21:43:42.027000+00:00,artificial_event
4,fb20e847-d2a1-4583-b874-393c9d1f3db9,2022-01-31 04:25:05.038000+00:00,artificial_event
...,...,...,...
406,f0704100-56e6-4117-ae4f-cf97e5d2d132,2021-12-19 07:32:14.004000+00:00,artificial_event
407,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 05:54:05.037000+00:00,artificial_event
408,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 04:10:51.041000+00:00,artificial_event
409,460a67ab-ee58-4279-b96d-ac56ac029d3e,2021-12-18 22:45:17.030000+00:00,artificial_event


In [17]:
arti_ign.isna().sum()

vehicleId    0
timestamp    0
type         0
dtype: int64

In [135]:
triggers.dtypes

Unnamed: 0                        int64
CTS           datetime64[ns, UTC-05:30]
PNID                              int64
NAME                             object
VAL                              object
dtype: object

In [154]:
triggers_v = pd.merge(left=triggers, right=pnid2[["ID", "PNID"]], left_on="PNID", right_on="PNID",how="inner")

In [157]:
triggers["PNID"]

0        206958332
1        206958332
2        206958332
3        206958332
4        206958332
           ...    
68665    193290358
68666    207925262
68667    207925262
68668    207925262
68669    207925262
Name: PNID, Length: 68670, dtype: object

In [155]:
triggers_v

,Unnamed: 0,CTS,PNID,NAME,VAL,ID


In [101]:
ignition_events = pd.DataFrame({
    "VEHICLE_ID":[],
    "TIMESTAMP":[],
    "Ignition_Event":[]
})

In [102]:
previous_status = "off"
for i in range(telemetry.shape[0]):
    tel = telemetry.iloc[i]
    if tel['IGNITION_STATUS']=="on" and previous_status=="off":
        ignition_events.loc[len(ignition_events.index)] = [tel["VEHICLE_ID"], tel["TIMESTAMP"], "ignitionOn"]
        previous_status = "on"
    elif tel['IGNITION_STATUS']=="off" and previous_status=="on":
        ignition_events.loc[len(ignition_events.index)] = [tel["VEHICLE_ID"], tel["TIMESTAMP"], "ignitionOff"]
        previous_status = "off"


In [149]:
previous_status = "OFF"
for i in range(triggers_v.shape[0]):
    tel = triggers_v.iloc[i]
    if tel["NAME"]!="IGN_CYL":continue
    if tel['VAL']=="ON" and previous_status=="OFF":
        print("Ignition on")
        ignition_events.loc[len(ignition_events.index)] = [np.NaN, tel["TIMESTAMP"], "ignitionOn"]
        previous_status = "ON"
    elif tel['VAL']=="OFF" and previous_status=="ON":
        print("Ignition on")
        ignition_events.loc[len(ignition_events.index)] = [np.NaN, tel["TIMESTAMP"], "ignitionOff"]
        previous_status = "OFF"

In [145]:
ignition_events

,VEHICLE_ID,TIMESTAMP,Ignition_Event
0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:38:57.002,ignitionOn
1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 04:18:05.031,ignitionOff
2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 04:16:31.002,ignitionOn
3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-10-03 02:51:53.035,ignitionOff
4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-10-03 03:02:52.029,ignitionOn
...,...,...,...
7561,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2023-01-20 22:23:48.040,ignitionOff
7562,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 02:54:20.020,ignitionOn
7563,cefc7df0-cf57-46ab-8180-55d437e07d9f,2023-01-21 03:06:54.029,ignitionOff
7564,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:08:17.020,ignitionOn
